# MMA Fighter Statistical Analysis  
This project aims to explore the relationship between fighter attributes and success metrics within MMA. By exploring various metrics such as striking accuracy, takedown defense, and physical attributes, we aim to determine what separates elite fighters from the rest.  


## Objectives
 
- Identify performance indicators that correlate with higher win percentages.  
- Analyze striking vs. grappling efficiency.  
- Determine if physical attributes influence fight outcomes.  


## Key Hypotheses
The following hypotheses will be tested using statistical analysis and visualizations:

* Fighters with a higher reach-to-height ratio tend to have a higher win percentage.


Does having a longer reach relative to height provide a striking advantage?



-  Fighters with a higher strike accuracy have a higher win percentage.

Does being more precise with strikes lead to increased success in fights?



- Do heavyweights have a lower win percentage than non-heavyweights?

Are heavyweight fighters less successful compared to lighter-weight fighters?



- Fighters with better takedown defense have a higher win percentage.


Does strong takedown defense contribute to more victories by keeping the fight standing?


- Fighters who absorb fewer strikes per minute have a higher win rate.

Does avoiding damage correlate with a higher chance of winning fights?



---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [10]:
import os
current_dir = os.getcwd()
current_dir

'c:\\Users\\seebi\\Downloads\\VSCODE_Projects'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [11]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [12]:
current_dir = os.getcwd()
current_dir

'c:\\Users\\seebi\\Downloads'

---

In [13]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as ex

file_path = r"C:\Users\Seebi\Downloads\VSCODE_Projects\Fighter-Statistical-Data/ufc-fighters-statistics-cleaned.csv"
df = pd.read_csv(file_path)

df.head()

# loading in the dataset, and looking through the first 5 rows


,name,wins,losses,draws,height_cm,weight_in_kg,reach_in_cm,stance,significant_strikes_landed_per_minute,significant_striking_accuracy,significant_strikes_absorbed_per_minute,significant_strike_defence,average_takedowns_landed_per_15_minutes,takedown_accuracy,takedown_defense,average_submissions_attempted_per_15_minutes,reach_to_height
0,Robert Drysdale,7,0,0,190.50,92.99,195.418730,Orthodox,0.00,0.0,0.00,0.0,7.32,100.0,0.0,21.9,1.025820
1,Daniel McWilliams,15,37,0,185.42,83.91,190.059538,Orthodox,3.36,77.0,0.00,0.0,0.00,0.0,100.0,21.6,1.025022
2,Dan Molina,13,9,0,177.80,97.98,182.029604,Orthodox,0.00,0.0,5.58,60.0,0.00,0.0,0.0,20.9,1.023789
3,Paul Ruiz,7,4,0,167.64,61.23,171.261548,Orthodox,1.40,33.0,1.40,75.0,0.00,0.0,100.0,20.9,1.021603
4,Collin Huckbody,8,2,0,190.50,83.91,193.040000,Orthodox,2.05,60.0,2.73,42.0,10.23,100.0,0.0,20.4,1.013333


---

In [14]:
import streamlit as st

from utils.data_processor import (
    load_and_process_data,
    get_fighters_by_weight_class,
    get_fighter_stats,
    get_comparison_metrics
)
from utils.visualization import create_spider_chart, create_stats_bar_chart

# Page config
st.set_page_config(
    page_title="UFC Fighter Statistics Dashboard",
    page_icon="🥊",
    layout="wide"
)

# Load custom CSS
with open('assets/styles.css') as f:
    st.markdown(f'<style>{f.read()}</style>', unsafe_allow_html=True)

# Load and process data
@st.cache_data
def load_data():
    return load_and_process_data('attached_assets/ufc-fighters-statistics-cleaned.csv')

df = load_data()

# Header
st.title("🥊 UFC Fighter Statistics Dashboard")
st.markdown("Compare fighters and analyze their statistics across different weight classes")

# Sidebar
st.sidebar.title("Fighter Selection")

# Gender selection
gender = st.sidebar.radio("Select Gender", ['Male', 'Female'])
filtered_df = df[df['gender'] == gender]

# Weight class selection
weight_classes_male = [
    'Flyweight', 'Bantamweight', 'Featherweight', 'Lightweight', 
    'Welterweight', 'Middleweight', 'Light Heavyweight', 'Heavyweight'
]
weight_classes_female = [
    'Strawweight', 'Flyweight', 'Bantamweight', 'Featherweight'
]

weight_class_options = weight_classes_male if gender == "Male" else weight_classes_female

weight_class = st.sidebar.selectbox("Select Weight Class", weight_class_options)

# Filter dataset based on weight class
filtered_df = filtered_df[filtered_df['weight_class'] == weight_class]

# Get fighters in selected weight class
fighters = get_fighters_by_weight_class(filtered_df, weight_class)

# Fighter selection
col1, col2 = st.columns(2)

with col1:
    fighter1 = st.selectbox("Select First Fighter", fighters, key='fighter1')
    fighter1_data = get_fighter_stats(filtered_df, fighter1)

with col2:
    remaining_fighters = [f for f in fighters if f != fighter1]
    fighter2 = st.selectbox("Select Second Fighter", remaining_fighters, key='fighter2')
    fighter2_data = get_fighter_stats(filtered_df, fighter2)

# Display fighter comparison
st.markdown("### Fighter Comparison")

# Basic stats comparison
stats_col1, stats_col2 = st.columns(2)

with stats_col1:
    st.markdown(f"#### {fighter1}")
    st.markdown(f"**Record:** {fighter1_data['wins']}-{fighter1_data['losses']}-{fighter1_data['draws']}")
    st.markdown(f"**Height:** {round(fighter1_data['height_cm'])} cm")
    st.markdown(f"**Weight:** {round(fighter1_data['weight_in_kg'])} kg")
    st.markdown(f"**Reach:** {round(fighter1_data['reach_in_cm'])} cm")
    st.markdown(f"**Stance:** {fighter1_data['stance']}")

with stats_col2:
    st.markdown(f"#### {fighter2}")
    st.markdown(f"**Record:** {fighter2_data['wins']}-{fighter2_data['losses']}-{fighter2_data['draws']}")
    st.markdown(f"**Height:** {round(fighter2_data['height_cm'])} cm")
    st.markdown(f"**Weight:** {round(fighter2_data['weight_in_kg'])} kg")
    st.markdown(f"**Reach:** {round(fighter2_data['reach_in_cm'])} cm")
    st.markdown(f"**Stance:** {fighter2_data['stance']}")

# Spider chart comparison
st.markdown("### Performance Comparison")

fighter1_metrics = get_comparison_metrics(fighter1_data)
fighter2_metrics = get_comparison_metrics(fighter2_data)

spider_chart = create_spider_chart(
    fighter1_metrics,
    fighter2_metrics,
    fighter1,
    fighter2
)
st.plotly_chart(spider_chart, use_container_width=True, key="spider")

# Detailed statistics
st.markdown("### Detailed Statistics")

detailed_col1, detailed_col2 = st.columns(2)

with detailed_col1:
    st.markdown(f"#### {fighter1} - Strike Statistics")
    strike_stats = {
        'Strikes Landed/Min': fighter1_data['significant_strikes_landed_per_minute'],
        'Strike Accuracy': fighter1_data['significant_striking_accuracy'],
        'Strikes Absorbed/Min': fighter1_data['significant_strikes_absorbed_per_minute'],
        'Strike Defense': fighter1_data['significant_strike_defence']
    }
    st.plotly_chart(create_stats_bar_chart(strike_stats, "Strike Statistics"), use_container_width=True, key=f"{fighter1}_stats")

with detailed_col2:
    st.markdown(f"#### {fighter2} - Strike Statistics")
    strike_stats = {
        'Strikes Landed/Min': fighter2_data['significant_strikes_landed_per_minute'],
        'Strike Accuracy': fighter2_data['significant_striking_accuracy'],
        'Strikes Absorbed/Min': fighter2_data['significant_strikes_absorbed_per_minute'],
        'Strike Defense': fighter2_data['significant_strike_defence']
    }
    st.plotly_chart(create_stats_bar_chart(strike_stats, "Strike Statistics"), use_container_width=True, key=f"{fighter2}_stats")

# Footer
st.markdown("---")
st.markdown("Data source: UFC Fighter Statistics Dataset")


ModuleNotFoundError: No module named 'utils'